In [ ]:
import os
import pandas as pd
from datetime import datetime
from math import log10
from scipy.stats import ecdf
from multiprocessing import Pool, cpu_count

In [ ]:
f = pd.read_csv("../data/pnwNP_modeledData/10378500.csv")

In [58]:
f.head(40)

,Unnamed: 0,streamflow_VIC,time,streamflow_PRMS,streamflow_NWM2d0,streamflow_NWM2d1,streamflow_NWIS,gage
0,0,0.767205,1950-01-01,12.308974,NaN,NaN,0.045307,10378500
1,1,1.181407,1950-01-02,17.368828,NaN,NaN,0.042475,10378500
2,2,1.245326,1950-01-03,15.873023,NaN,NaN,0.042475,10378500
3,3,1.250622,1950-01-04,13.633054,NaN,NaN,0.045307,10378500
4,4,1.247074,1950-01-05,11.714673,NaN,NaN,0.048139,10378500
5,5,1.244287,1950-01-06,10.202055,NaN,NaN,0.053802,10378500
6,6,1.246358,1950-01-07,8.989211,NaN,NaN,0.053802,10378500
7,7,1.259414,1950-01-08,8.002232,NaN,NaN,0.053802,10378500
8,8,1.246337,1950-01-09,7.188663,NaN,NaN,0.053802,10378500
9,9,1.442424,1950-01-10,6.739753,NaN,NaN,0.050970,10378500


In [74]:


df = f[["time","streamflow_NWIS"]]
df = df.rename(columns = {"time": "Date", "streamflow_NWIS": "q"})
df["Date"] = pd.to_datetime(df["Date"])
df = df[['Date', 'q']].dropna()
df['q'] = df['q'].round(1)
df['q_peak'] = df['q'].where(df['q'] > df['q'].quantile(0.25), 0)

df['slp_b'] = (df['q_peak'] - df['q_peak'].shift()) / (df['Date'] - df['Date'].shift()).dt.days
df['slp_f'] = (df['q_peak'].shift(-1) - df['q_peak']) / (df['Date'].shift(-1) - df['Date']).dt.days

df['peak_flag'] = (df['slp_b'] > 0.0001) & (df['slp_f'] < 0)
df['peak_flag'] = df['peak_flag'].fillna(0)

df['event_id'] = df['peak_flag'].cumsum() + 1
df['nf_start'] = (df['q'] == 0) & (df['q'].shift() != 0)

In [78]:
df[df["nf_start"]==True]

,Date,q,q_peak,slp_b,slp_f,peak_flag,event_id,nf_start
0,1950-01-01,0.0,0.0,NaN,0.0,False,1,True
10,1950-01-11,0.0,0.0,0.0,0.0,False,1,True
213,1950-08-02,0.0,0.0,0.0,0.0,False,18,True
221,1950-08-10,0.0,0.0,0.0,0.0,False,18,True
284,1950-10-12,0.0,0.0,0.0,0.0,False,18,True
...,...,...,...,...,...,...,...,...
15004,1991-01-30,0.0,0.0,0.0,0.0,False,816,True
15017,1991-02-12,0.0,0.0,0.0,0.0,False,816,True
15032,1991-02-27,0.0,0.0,0.0,0.0,False,816,True
15170,1991-07-15,0.0,0.0,0.0,0.0,False,826,True


In [85]:
# def recession_fun(m):
m=18
t = df[df['event_id'] == m].copy()
# t['nf_start'].fillna(0, inplace=True)
t = t.fillna({"nf_start":0},inplace=True)

if t['nf_start'].sum() != 0 and t['q'].iloc[0] != 0 and t['q_peak'].sum() != 0:
    print(1)
    t['dry_event_id'] = t['nf_start'].cumsum()
    t['dry_event_id'] = t['dry_event_id'].where(t['q'] > 0, 0)
    
    dry_date = t[t['dry_event_id'] > 0].groupby('dry_event_id').agg({'Date': 'min'}).reset_index()
    dry_date = dry_date.sort_values(by=['n', 'date'], ascending=[False, True]).iloc[0]['Date']
    
    t = t[t['Date'] <= dry_date]
    
    event_id = t['event_id'].iloc[0]
    peak_date = datetime.strptime(t['Date'].iloc[0], '%Y-%m-%d').timetuple().tm_yday
    peak_value = t['q'].iloc[0]
    peak_quantile = ecdf(df['q'])(peak_value)
    peak2zero = len(t)
    
    t['dQ'] = t['q'].shift() - t['q']
    t = t[t['dQ'] >= 0]
    
    model = pd.DataFrame({'log10_dQ': log10(t['dQ'] + 0.1), 'log10_q': log10(t['q'] + 0.1)})
    model = model.dropna()
    
    drying_rate = model['log10_q'].iloc[1]
    p_value = model['log10_dQ'].iloc[1]
    
    output = pd.DataFrame({'event_id': [event_id], 
                           'peak_date': [peak_date], 
                           'peak_value': [peak_value], 
                           'peak_quantile': [peak_quantile], 
                           'peak2zero': [peak2zero], 
                           'drying_rate': [drying_rate], 
                           'p_value': [p_value]})
else:
    output = pd.DataFrame({'event_id': [t['event_id'].iloc[0]],
                           'peak_date': [None], 'peak_value': [None],
                           'peak_quantile': [None],
                           'peak2zero': [None],
                           'drying_rate': [None],
                           'p_value': [None]})


1


KeyError: 'n'

In [102]:
m=18
t = df[df['event_id'] == m].copy()
# t = t['nf_start'].fillna(0)
# t = tfillna({"nf_start":0},inplace=True)

In [103]:
t

,Date,q,q_peak,slp_b,slp_f,peak_flag,event_id,nf_start
190,1950-07-10,0.4,0.4,0.1,-0.1,True,18,False
191,1950-07-11,0.3,0.3,-0.1,0.0,False,18,False
192,1950-07-12,0.3,0.3,0.0,0.0,False,18,False
193,1950-07-13,0.3,0.3,0.0,-0.1,False,18,False
194,1950-07-14,0.2,0.2,-0.1,-0.2,False,18,False
...,...,...,...,...,...,...,...,...
297,1950-10-25,0.1,0.0,0.0,0.0,False,18,False
298,1950-10-26,0.1,0.0,0.0,0.0,False,18,False
299,1950-10-27,0.1,0.0,0.0,0.0,False,18,False
300,1950-10-28,0.1,0.0,0.0,0.2,False,18,False


In [88]:
    dry_date = t[t['dry_event_id'] > 0].groupby('dry_event_id').agg({'Date': 'min'}).reset_index()
    dry_date = dry_date.sort_values(by=['n', 'date'], ascending=[False, True]).iloc[0]['Date']

,dry_event_id,Date
0,1,1950-08-04
1,2,1950-10-05
2,3,1950-10-19


In [92]:
t.head(50)

,Date,q,q_peak,slp_b,slp_f,peak_flag,event_id,nf_start,dry_event_id
190,1950-07-10,0.4,0.4,0.1,-0.1,True,18,False,0
191,1950-07-11,0.3,0.3,-0.1,0.0,False,18,False,0
192,1950-07-12,0.3,0.3,0.0,0.0,False,18,False,0
193,1950-07-13,0.3,0.3,0.0,-0.1,False,18,False,0
194,1950-07-14,0.2,0.2,-0.1,-0.2,False,18,False,0
195,1950-07-15,0.1,0.0,-0.2,0.0,False,18,False,0
196,1950-07-16,0.1,0.0,0.0,0.0,False,18,False,0
197,1950-07-17,0.1,0.0,0.0,0.0,False,18,False,0
198,1950-07-18,0.1,0.0,0.0,0.0,False,18,False,0
199,1950-07-19,0.1,0.0,0.0,0.0,False,18,False,0


In [ ]:
files = [f for f in os.listdir('data/daily_data_with_ climate_and_PET/csv') if f.endswith('.csv')]

In [ ]:
def metrics_fun(n):

    
    df = f[["time","streamflow_NWIS"]]
    df = df.rename(columns = {"time": "Date", "streamflow_NWIS": "q"})
    df["Date"] = pd.to_datetime(df["Date"])
    df = df[['Date', 'q']].dropna()
    df['q'] = df['q'].round(1)
    df['q_peak'] = df['q'].where(df['q'] > df['q'].quantile(0.25), 0)
    
    df['slp_b'] = (df['q_peak'] - df['q_peak'].shift()) / (df['Date'] - df['Date'].shift()).dt.days
    df['slp_f'] = (df['q_peak'].shift(-1) - df['q_peak']) / (df['Date'].shift(-1) - df['Date']).dt.days
    
    df['peak_flag'] = (df['slp_b'] > 0.0001) & (df['slp_f'] < 0)
    df['peak_flag'] = df['peak_flag'].fillna(0)
    
    df['event_id'] = df['peak_flag'].cumsum() + 1
    df['nf_start'] = (df['q'] == 0) & (df['q'].shift() != 0)



    
    def recession_fun(m):
        t = df[df['event_id'] == m].copy()
        t['nf_start'].fillna(0, inplace=True)
        
        if t['nf_start'].sum() != 0 and t['q'].iloc[0] != 0 and t['q_peak'].sum() != 0:
            t['dry_event_id'] = t['nf_start'].cumsum()
            t['dry_event_id'] = t['dry_event_id'].where(t['q'] > 0, 0)
            
            dry_date = t[t['dry_event_id'] > 0].groupby('dry_event_id').agg({'date': 'min'}).reset_index()
            dry_date = dry_date.sort_values(by=['n', 'date'], ascending=[False, True]).iloc[0]['date']
            
            t = t[t['date'] <= dry_date]
            
            event_id = t['event_id'].iloc[0]
            peak_date = datetime.strptime(t['date'].iloc[0], '%Y-%m-%d').timetuple().tm_yday
            peak_value = t['q'].iloc[0]
            peak_quantile = ecdf(df['q'])(peak_value)
            peak2zero = len(t)
            
            t['dQ'] = t['q'].shift() - t['q']
            t = t[t['dQ'] >= 0]
            
            model = pd.DataFrame({'log10_dQ': log10(t['dQ'] + 0.1), 'log10_q': log10(t['q'] + 0.1)})
            model = model.dropna()
            
            drying_rate = model['log10_q'].iloc[1]
            p_value = model['log10_dQ'].iloc[1]
            
            output = pd.DataFrame({'event_id': [event_id], 
                                   'peak_date': [peak_date], 
                                   'peak_value': [peak_value], 
                                   'peak_quantile': [peak_quantile], 
                                   'peak2zero': [peak2zero], 
                                   'drying_rate': [drying_rate], 
                                   'p_value': [p_value]})
        else:
            output = pd.DataFrame({'event_id': [t['event_id'].iloc[0]],
                                   'peak_date': [None], 'peak_value': [None],
                                   'peak_quantile': [None],
                                   'peak2zero': [None],
                                   'drying_rate': [None],
                                   'p_value': [None]})
        
        return output
    
    def dry_fun(m):
        t = df[df['event_id'] == m].copy()
        t['nf_start'].fillna(0, inplace=True)
        
        if t['nf_start'].sum() != 0:
            t['dry_event_id'] = t['nf_start'].cumsum()
            t['dry_event_id'] = t['dry_event_id'].where(t['q'] > 0, 0)
            
            dry_event = t[t['dry_event_id'] > 0].groupby('dry_event_id').agg({'date': 'min'}).reset_index()
            dry_event = dry_event.sort_values(by=['n', 'date'], ascending=[False, True]).iloc[0]['dry_event_id']
            
            t = t[t['dry_event_id'] == dry_event]
            
            output = pd.DataFrame({'event_id': [t['event_id'].iloc[0]],
                                   'calendar_year': [t['date'].iloc[0].year],
                                   'season': ['Winter' if t['date'].iloc[0].month <= 3 else 'Spring' if t['date'].iloc[0].month <= 6 else 'Summer' if t['date'].iloc[0].month <= 9 else 'Fall'],
                                   'meteorologic_year': [t['date'].iloc[0].year - 1 if t['season'].iloc[0] == 'Winter' else t['date'].iloc[0].year],
                                   'dry_date_start': [datetime.strptime(t['date'].iloc[0], '%Y-%m-%d').timetuple().tm_yday],
                                   'dry_date_mean': [t['date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').timetuple().tm_yday).mean()],
                                   'dry_dur': [len(t)]})
        else:
            output = pd.DataFrame({'event_id': [t['event_id'].iloc[0]],
                                   'calendar_year': [None],
                                   'season': [None], 
                                   'meteorologic_year': [None],
                                   'dry_date_start': [None],
                                   'dry_date_mean': [None],
                                   'dry_dur': [None]})
        
        return output

    
    metrics = pd.concat([recession_fun(m) for m in range(1, df['event_id'].max() + 1)]).reset_index(drop=True)
    metrics = pd.concat([metrics, dry_fun(m) for m in range(1, df['event_id'].max() + 1)]).reset_index(drop=True)
    metrics['gage'] = gage
    metrics = metrics.dropna(subset=['dry_dur'])
    
    return metrics

In [ ]:
def execute(a):
    try:
        return metrics_fun(a)
    except Exception as e:
        return pd.DataFrame({'event_id': [None],
                             'peak_date': [None],
                             'peak2zero': [None],
                             'drying_rate': [None],
                             'calendar_year': [None],
                             'season': [None],
                             'meteorologic_year': [None],
                             'dry_date_start': [None],
                             'dry_date_mean': [None],
                             'dry_dur': [None],
                             'p_value': [None], 
                             'gage': [os.path.splitext(files[a])[0]]})

if __name__ == '__main__':
    t0 = datetime.now()
    
    with Pool(cpu_count()) as p:
        output = pd.concat(p.map(execute, range(len(files))))
    
    output.to_csv('./data/metrics_by_event.csv', index=False)
    
    tf = datetime.now()
    tf - t0